# Interactive Portfolio Dashboard

This notebook creates an interactive dashboard to visualize portfolio performance and stress testing scenarios.  
We use the Dash framework along with Plotly for interactive visualizations.

**Dashboard Features:**
- **Cumulative Returns Chart:** Displays the portfolio's cumulative returns over time.
- **Scatter Matrix:** Visualizes relationships between asset returns.
- **Stress Scenario Selector:** A dropdown to choose among various stress testing scenarios.
- **Stress Testing Visualization:** Displays Monte Carlo simulation paths for the selected scenario.

In [ ]:
#if not already installed
!pip install dash dash-bootstrap-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
#importing required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

print("Dashboard libraries imported successfully!")


Dashboard libraries imported successfully!


In [ ]:

from google.colab import files

# Upload the processed CSV file (e.g., historical_prices_cleaned.csv)
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the cleaned price data
prices = pd.read_csv(filename, index_col=0, parse_dates=True)
print("Cleaned Price Data (first 5 rows):")
print(prices.head())

def compute_returns(prices):
    returns = prices.pct_change(fill_method=None).dropna()
    return returns

returns = compute_returns(prices)
cum_returns = (1 + returns).cumprod()
print("Cumulative Returns (first 5 rows):")
print(cum_returns.head())


Saving historical_prices_cleaned.csv to historical_prices_cleaned.csv
Cleaned Price Data (first 5 rows):
              AMZN         GE      GOOGL        HSY        MMM       MSFT  \
Date                                                                        
2012-01-01  9.1305  69.240280  16.208012  44.837509  46.040127  22.132902   
2012-01-08  8.9210  69.945709  15.583897  45.421150  46.167137  22.243130   
2012-01-15  9.5465  71.096581  14.611446  45.465488  47.299232  23.392693   
2012-01-22  9.7685  70.651070  14.461590  45.243847  48.298786  23.014755   
2012-01-29  9.3840  70.613945  14.869270  45.184734  48.447895  23.809998   

                  SHY  
Date                   
2012-01-01  71.186943  
2012-01-08  71.245903  
2012-01-15  71.229073  
2012-01-22  71.304886  
2012-01-29  71.237541  
Cumulative Returns (first 5 rows):
                AMZN        GE     GOOGL       HSY       MMM      MSFT  \
Date                                                                     
2012

In [ ]:

# Prepare cumulative returns chart
cum_returns_reset = cum_returns.reset_index()
# Ensure the date column is named "Date"
cum_returns_reset.rename(columns={"index": "Date"}, inplace=True)

fig_cum = px.line(
    cum_returns_reset,
    x="Date",
    y=cum_returns.columns,
    title="Cumulative Returns of Portfolio",
    labels={"Date": "Date", "value": "Cumulative Return"}
)
fig_cum.update_layout(title_font_size=24, xaxis_title="Date", yaxis_title="Cumulative Return")

# Prepare scatter matrix of asset returns
label_dict = {
    "AMZN": "Amazon",
    "GE": "GE",
    "GOOGL": "Google",
    "HSY": "Hershey",
    "MMM": "3M",
    "MSFT": "Microsoft",
    "SHY": "SHY"
}

fig_scatter = px.scatter_matrix(
    returns.reset_index(),
    dimensions=returns.columns,
    labels=label_dict,
    title="Scatter Matrix of Asset Returns"
)

# Make the figure bigger so labels are more spaced out
fig_scatter.update_layout(
    width=1200,
    height=1000,
    title_font_size=24
)

# Reduce label overlap by rotating x-axis tick labels
fig_scatter.update_xaxes(tickangle=45, automargin=True)
fig_scatter.update_yaxes(automargin=True)

fig_scatter.show()

In [ ]:

def monte_carlo(returns, scale_returns=1.0, scale_cov=1.0, num_sim=500, horizon=52):

    np.random.seed(42)
    mean_ret = returns.mean() * scale_returns
    cov_matrix = returns.cov() * scale_cov
    sims = []
    for _ in range(num_sim):
        sim = np.random.multivariate_normal(mean_ret, cov_matrix, horizon)
        sim_cum = (sim + 1).cumprod(axis=0)
        sims.append(sim_cum)
    return np.array(sims)

scenarios = {
    "Baseline": {"scale_returns": 1.0, "scale_cov": 1.0},
    "Market Crash": {"scale_returns": 0.5, "scale_cov": 1.0},
    "High Volatility": {"scale_returns": 1.0, "scale_cov": 2.0},
    "Combined Stress": {"scale_returns": 0.5, "scale_cov": 2.0}
}

simulations_dict = {}
for key, vals in scenarios.items():
    simulations_dict[key] = monte_carlo(
        returns,
        scale_returns=vals["scale_returns"],
        scale_cov=vals["scale_cov"],
        num_sim=500,
        horizon=52
    )

print("Stress testing simulations computed for scenarios:", list(simulations_dict.keys()))


Stress testing simulations computed for scenarios: ['Baseline', 'Market Crash', 'High Volatility', 'Combined Stress']


In [ ]:
# %% [code]
# Create a Dash app using the standard dash.Dash class
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the app layout
app.layout = html.Div([
    html.H1("Interactive Portfolio Dashboard", style={"textAlign": "center"}),

    dbc.Row([
        dbc.Col(dcc.Graph(id="cum_returns_graph", figure=fig_cum), md=6),
        dbc.Col(dcc.Graph(id="scatter_matrix_graph", figure=fig_scatter), md=6)
    ], style={"marginTop": "20px"}),

    html.Br(),

    html.Div([
        html.Label("Select Stress Scenario:"),
        dcc.Dropdown(
            id="stress_scenario",
            options=[
                {"label": "Baseline", "value": "Baseline"},
                {"label": "Market Crash", "value": "Market Crash"},
                {"label": "High Volatility", "value": "High Volatility"},
                {"label": "Combined Stress", "value": "Combined Stress"}
            ],
            value="Baseline",
            style={"width": "300px"}
        )
    ], style={"marginBottom": "20px", "textAlign": "center"}),

    dbc.Row([
        dbc.Col(dcc.Graph(id="stress_returns_graph"), md=12)
    ])
])

# Callback to update the stress scenario graph based on dropdown selection
@app.callback(
    Output("stress_returns_graph", "figure"),
    Input("stress_scenario", "value")
)
def update_stress_graph(selected_scenario):
    sims = simulations_dict[selected_scenario]
    fig = go.Figure()
    # Plot cumulative returns for the first asset from the first 50 simulations
    for sim in sims[:50]:
        fig.add_trace(go.Scatter(
            x=list(range(sim.shape[0])),
            y=sim[:, 0],
            mode="lines",
            line=dict(color="gray", width=1),
            opacity=0.3,
            showlegend=False
        ))
    fig.update_layout(
        title=f"{selected_scenario} Scenario: Cumulative Returns (First Asset)",
        xaxis_title="Weeks",
        yaxis_title="Cumulative Return",
        title_font_size=24
    )
    return fig

# Run the dashboard server
app.run_server(debug=True, port=8050)


<IPython.core.display.Javascript object>